In [1]:
%pylab inline
import pandas as pd
import keras
import kapre
import glob
import arrow
import pathlib
from tqdm import tqdm
from soph import center_wave, MFCC

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [2]:
base_path = "submissions/"+arrow.now().to('US/Eastern').format('YYYY-MM-DD/HH-mm')
base_path

'submissions/2018-01-07/01-31'

In [3]:
base_path = pathlib.Path(base_path)
base_path.mkdir(parents=True, exist_ok=True)

In [4]:
base_path.joinpath("model.hdf5")

PosixPath('submissions/2018-01-07/01-31/model.hdf5')

In [5]:
model_path = pathlib.Path("logs/cnn/2018-01-06-23-16/model-checkpoint.hdf5")

#move model 
model_path.rename(pathlib.Path(base_path / "model.hdf5"))
model_path = base_path / "model.hdf5"

In [6]:
print(model_path)

submissions/2018-01-07/01-31/model.hdf5


In [7]:
ex_df = pd.read_pickle("data/ex_df.pkl")

In [8]:
test_model = keras.models.load_model(
    model_path,
    custom_objects={
        "Melspectrogram": kapre.time_frequency.Melspectrogram,
        "AdditiveNoise": kapre.augmentation.AdditiveNoise,
        "MFCC": kapre.time_frequency.MFCC,
        "DeltaDelta": kapre.utils.DeltaDelta,
    })

In [9]:
label_dict = {}
for label_i in ex_df[-ex_df.label_i.isna()].label_i.unique():
    
    label_dict[int(label_i)] = ex_df[ex_df.label_i==label_i].label.mode().values[0]

In [10]:
test_df = ex_df.loc[ex_df.state=="submission",:].copy()
test_df.reset_index(inplace=True)
num_rows = len(test_df)

In [11]:
batch_size = 512
y_pred = []
for i in tqdm(np.arange(0,num_rows,batch_size)):
    
    curr_size = min(batch_size,num_rows-i)
    x = zeros((curr_size,1,16000))
    for x_i,j in enumerate(range(i,min(i+batch_size, num_rows))):
        
        x[x_i,...] = center_wave(test_df.fn[j])
        
    pred_class = np.argmax(test_model.predict_on_batch(x), axis=1)
    y_pred.append(pred_class[:,np.newaxis])

100%|██████████| 310/310 [02:13<00:00,  2.33it/s]


In [12]:
y_pred= np.concatenate(y_pred)

In [13]:
len(y_pred)

158538

In [14]:
len(test_df)

158538

In [15]:
test_df.loc[:,"y_pred_raw"] = y_pred.flat

In [16]:
test_df["label"] = test_df.y_pred_raw.map(label_dict)

In [17]:
test_df["fname"] = test_df.fn.str.split("/").str[-1]

In [18]:
submission_df = test_df[["fname","label"]]

In [19]:
submission_df.to_csv(base_path / "submission.csv",index=False)